<a href="https://colab.research.google.com/github/DaeSeokSong/Aengmu/blob/main/Aengmu_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ERROR 4] Korean(Hangul) breaking phenomenon Solution on Colab

In [ ]:
""" 
ERROR 4. plot "Korean" breaking phenomenon
- Solution: Installing(↓) and setting(plt.rc('font', family='NanumBarunGothic')) Nanum font, after runtime restart
"""
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# Google Drive Local Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Morpheme analyzer
*   [Reference](https://soohee410.github.io/compare_tagger)
*   [Install](https://sanghyu.tistory.com/170)

In [ ]:
# okt, komoran, kkma
# install konlpy (okt, komoran, kkma)
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

# mecab (take a long time)
# set env
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

# install konlpy (mecab)
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

# Error Solution

1. TypeError: startJVM() got an unexpected keyword argument 'convertStrings' [(JVM)](https://gyulogs.tistory.com/130)
2. NameError: name 'Tagger' is not defined [(Mecab)](https://sosomemo.tistory.com/31)
3. ParserError: Error tokenizing data. C error [(Pandas)](https://mskim8717.tistory.com/82)
4. plot "Korean" breaking phenomenon on Colab [(Matplotlib)](https://teddylee777.github.io/colab/colab-korean)

In [ ]:
""" 
ERROR 1. TypeError: startJVM() got an unexpected keyword argument 'convertStrings'
- Solution: /usr/local/lib/python3.7/dist-packages/konlpy/jvm.py, 67 line (convertStrings=True) comments processing and save jvm.py before import pakage
"""

""" 
ERROR 2. NameError: name 'Tagger' is not defined 
- Solution: Execute mecab.sh script (under code excute)
"""
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# Import
*   [KoNLPy Reperence](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)



In [ ]:
# import for MechineLearning
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import metrics, losses, callbacks
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

# import Morpheme analyzer
from konlpy.tag import Kkma, Komoran, Okt, Mecab
from konlpy.utils import pprint

# import etc
from google.colab import files
import time
import os

# Change run location
%cd /content/drive/My Drive/DeepLearning/PROJECT_AON
# Matplotlib set font on NanumBarunGothic
plt.rc('font', family='NanumBarunGothic')
# Numpy print set
np.set_printoptions(linewidth=200, precision=2)

#Grobal Variable
*   [Concept Reference](https://reniew.github.io/25/)

In [ ]:
"""
ranking about time spent morpheme analyzing (The fastest is number one.)

1. Mecab
2. Komoran
3. Okt
4. Kkma

Mecab is faster than Kkma about 30~40 times
Mecab is faster than Okt about 10 times
Mecab is faster than Komoran about 5 times
"""

# Early versions use macab.
MECAB = Mecab()

# Learning target's name
AI_TARGET_NAME = "대석"

# Funtion

Data Prepare Functions

In [ ]:
# Extract reply time (minute)
def extract_replytime(content) :
    idx_list1 = [i for i, value in enumerate(content) if value == ']']
    idx_list2 = [i for i, value in enumerate(content) if value == '[']

    end_idx = idx_list1[1]
    start_idx = idx_list2[1] + 1

    replytime = content[start_idx : end_idx]

    if replytime[:2] == "오전" :
        replytime = (int(replytime[2:replytime.index(":")]) * 60) + int(replytime[replytime.index(":") + 1 :])
    else :
        replytime = 60 * 12 + (int(replytime[2:replytime.index(":")]) * 60) + int(replytime[replytime.index(":") + 1 :])

    return replytime

# Comparison about reply time
def compare_replytime(pre_text, cur_text) :
    pre_time = extract_replytime(pre_text)
    cur_time = extract_replytime(cur_text)

    result = False
    if cur_time - pre_time >= 10 : result = True

    return result

# Extract Data about kakao talk's content
def extract_data(X_dataset, y_dataset, talk_list) :
    global AI_TARGET_NAME

    before_respondent = ""
    for talk in talk_list :
        for texts in talk.values :
            tmp_X_texts = []
            tmp_Y_texts = []
            texts = texts.tolist()
            for idx, text in enumerate(texts) : 
                # Don't extract email and date
                if (not "저장한 날짜" in text) and (not "--------------" in text) and (not "@" in text) :
                    if not "]" in text : 
                        if before_respondent == AI_TARGET_NAME : tmp_Y_texts.append(text)
                        else : tmp_X_texts.append(text)

                        continue
                    else : cur_respondent = text[1:text.index("]")]

                    if (":" in text) and (before_respondent != "") :
                        if compare_replytime(texts[idx - 1], text) or cur_respondent != before_respondent:
                            if len(tmp_X_texts) != 0 : X_dataset.append(tmp_X_texts)
                            if len(tmp_Y_texts) != 0 : y_dataset.append(tmp_Y_texts)
                            tmp_X_texts = []
                            tmp_Y_texts = []

                    if AI_TARGET_NAME in text :
                        ptext = text[(text.rfind(']') + 2) : len(text)]
                        if ptext.find("http") == -1 and ptext != "사진" and ptext != "" : 
                            before_respondent = AI_TARGET_NAME
                            if ptext != '' : tmp_Y_texts.append(ptext)
                    elif "]" in text :
                        ptext = text[(text.rfind(']') + 2) : len(text)]
                        if ptext.find("http") == -1 and ptext != "사진" and ptext != "" : 
                            before_respondent = cur_respondent
                            if ptext != '' : tmp_X_texts.append(ptext)

                    if idx == len(texts) - 1 :
                        if len(tmp_X_texts) != 0 : X_dataset.append(tmp_X_texts)
                        if len(tmp_Y_texts) != 0 : y_dataset.append(tmp_Y_texts)

Data Pre-processing functions

In [ ]:
# Auto calibrate spacing functions
def auto_spacing(X_dataset, y_dataset) :
    calibrate_spcaing(X_dataset)
    calibrate_spcaing(y_dataset)

def calibrate_spcaing(context_dataset) :
    global MECAB

    for idx, texts in enumerate(context_dataset) :
        for repairIdx, text in enumerate(texts) :
            analyzedRes = MECAB.pos(text)
            for mecabR in analyzedRes :
                # "MAG" == Adverb
                if mecabR[1] == "MAG" :
                    try :
                        startIdx = text.index(mecabR[0])
                        if len(mecabR[0]) == 1 :
                            if text[startIdx + 1] != " " and text[startIdx + 1].isalnum() : 
                                if text[startIdx + 2] != " " :
                                    text = text[:startIdx] + text[startIdx] + " " + text[startIdx + 1 :]
                                    context_dataset[idx][repairIdx] = text
                        else : 
                            if text[startIdx + len(mecabR[0])] != " " and text[startIdx + len(mecabR[0])].isalnum() : 
                                if text[startIdx + len(mecabR[0]) + 1] != " " :
                                    text = text[:startIdx] + text[startIdx : startIdx + len(mecabR[0])] + " " + text[startIdx + len(mecabR[0]) :]
                                    context_dataset[idx][repairIdx] = text
                    except IndexError :
                        continue

# Extraction 1 word by 1 content (use in Early version Model)
def extract_1word(X_dataset, y_dataset) :
    """ If don't use list, data type is str. this is caused by need just one word on word2vec """
    for idx in range(0, len(X_dataset)) : X_dataset[idx] = X_dataset[idx][0].split()
    for idx in range(0, len(y_dataset)) : y_dataset[idx] = y_dataset[idx][0].split()

# Delete stopword in sentence
def stopword_filtering(X_dataset, y_dataset) :
    delete_stopword(X_dataset)
    delete_stopword(y_dataset)

def delete_stopword(word_dataset) :
    global MECAB

    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    for idx in range(0, len(word_dataset)) :
        #print(word_dataset[idx])
        tokenized_data = MECAB.morphs(word_dataset[idx][0])
        tokenized_data = [word for word in tokenized_data if not word in stopwords]
        word_dataset[idx] = tokenized_data

# If x and y size different, equalize that
def equalize_size(X_dataset, y_dataset) :
    if len(X_dataset) > len(y_dataset) : X_dataset = X_dataset[1 : len(y_dataset) + 1]
    elif len(y_dataset) > len(X_dataset) : y_dataset = y_dataset[1 : len(X_dataset) + 1]

    return X_dataset, y_dataset
    
# Reshaping size on ndarray for training
def reshape_sizing(X_dataset, y_dataset) :
    X_dataset = X_dataset.reshape(X_sizing(X_dataset))
    y_dataset = y_dataset.reshape(y_sizing(y_dataset))

    return X_dataset, y_dataset

def X_sizing(X_dataset) :
    common_factors = factorization(X_dataset.shape[0]) # 공약수
    step_size = common_factors[0] # timesteps
    vector_length = common_factors[1] # n-length vectors

    return int((X_dataset.shape[0] / step_size) / vector_length), step_size, vector_length

def y_sizing(y_dataset) :
    common_factors = factorization(y_dataset.shape[0])
    dim_2 = common_factors[0] * common_factors[1]

    return int(y_dataset.shape[0] / dim_2), dim_2

# Factorization for reshaping's variable
def factorization(x) :
    common_factors = []
    d = 2 
    
    while d <= x: 
        if x % d == 0: 
            common_factors.append(d)
            x = x / d 
        else: d = d + 1
    
    return common_factors

# reshape_sizing 메서드 수정해야함 (하는 중)
# 빈도수 너무 높거나 낮은건 제외하는 메서드 추가해야됌 (다음에 할 거)

Word2Vec Functions (Distributed Representation)
*   [Word2Vec Reference 1](https://ebbnflow.tistory.com/153)
*   [Word2Vec Reference 2](https://monetd.github.io/python/nlp/Word-Embedding-Word2Vec-%EC%8B%A4%EC%8A%B5/#%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec-%EB%A7%8C%EB%93%A4%EA%B8%B0)
*   [PCA(sklearn) Reference](https://m.blog.naver.com/tjdrud1323/221720259834)

In [ ]:
# Draw a two-dimensional graph by entering the words, values of the two-dimensional X-axis, and values of the Y-axis.
def plot_2d_graph(vocabs, xs, ys):
    plt.figure(figsize=(8 ,6))
    plt.scatter(xs, ys, marker = 'o')
    for i, v in enumerate(vocabs):
        plt.annotate(v, xy=(xs[i], ys[i]))

# Conversioning dataset to word2vec format
def dataset_to_word2vec(X_dataset, y_dataset) :
    return word2vec(X_dataset, 'X'), word2vec(y_dataset, 'y')

def word2vec(word_dataset, T) :
    # Init words and vectors
    w2v = Word2Vec(word_dataset, size=100, window=3, min_count=1, workers=6, sg=1)

    # Set word vectors
    word_vectors = w2v.wv
    vocabs = word_vectors.vocab.keys()
    word_vector_list = [word_vectors[v] for v in vocabs]

    # Confirm word similarity
    # if "경호" in vocabs : print(word_vectors.most_similar("경호"))
    # else : print(word_vectors.most_similar("대석"))
    
    pca = PCA(n_components=2)
    xys = pca.fit_transform(word_vector_list)
    xs = xys[:,0]
    ys = xys[:,1]

    # plot_2d_graph(vocabs, xs, ys)
    return xys.reshape(-1)

# vec2word 함수 만들어야 함 (다다음에 할 거)

#Main

Prepare Data

In [ ]:
""" 
ERROR 3. ParserError: Error tokenizing data. C error 
- Solution: add code in read_csv = , sep='\t'
"""
# uploaded_files = files.upload()

file_list = os.listdir('./')
fs = [f for f in file_list if f.endswith('.txt')]

talk_list = []
for f in fs :
    talk_list.append(pd.read_csv(f, sep='\t'))

# Init Dataset
X_dataset = []
y_dataset = []

# Data extraction
extract_data(X_dataset, y_dataset, talk_list)

Pre-processing Data
*  [Dataset Classification Reference 1](https://ganghee-lee.tistory.com/38)
*  [Dataset Classification Reference 2](https://ysyblog.tistory.com/69)

In [ ]:
# Space Modification
auto_spacing(X_dataset, y_dataset)

# Early version is used y_list[][0] and x_list[][0] (1 word) by learning model
extract_1word(X_dataset, y_dataset)

# Stopword deleted
stopword_filtering(X_dataset, y_dataset)

# Size equalize
X_dataset, y_dataset = equalize_size(X_dataset, y_dataset)

# Data type change on np.ndarray
X_dataset = np.asarray(X_dataset).astype(object)
y_dataset = np.asarray(y_dataset).astype(object)

# Word2Vec on dataset
X_dataset, y_dataset = dataset_to_word2vec(X_dataset, y_dataset)

# Dataset reshape
X_dataset, y_dataset = reshape_sizing(X_dataset, y_dataset)

# Data division (Train : Validation : Test = 6 : 2 : 2)
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset,
                                                    test_size = 0.2)
print("########## Train + Validation (X,) (y,) / Test (X,) (y,) ##########")
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size = 0.2)
print("########## Train (X,) (y,) / Validation (X,) (y,) ##########")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

Modeling

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation="relu", padding='same', input_shape=(X_train.shape[1:])))
model.add(MaxPooling1D(pool_size=2, paddig='same'))
model.add(Conv1D(filters=64, kernel_size=3, activation="relu", padding='same'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Conv1D(filters=64, kernel_size=3, activation="relu", padding='same'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Flatten())
model.add(Dense(64, activation="relu")) # Performance improves once it passes through layers that have been amplified datas
model.add(Dense(1, activation="softmax")) # output data == n( == y_dataset.shape[1]) things
model.compile(
    loss=losses.CategoricalCrossentropy(), # loss function for NLP is categorical_crossentropy(keras.losses) << personally
    optimizer=Adam(), # pram ex. learning_rate=0.0001
    metrics=[metrics.CategoricalAccuracy()] # accuracy for NLP is categorical_accuracy(keras.metrics) << personally
)

model.summary()

Training Model

In [ ]:
history = model.fit(X_train, y_train,
          validation_data = (X_val, y_val),
          batch_size = 4, # Big == good (but enough)
          epochs = 80,
          callbacks=[ # helper for learning
                     callbacks.EarlyStopping(monitor='binary_accuracy', mode='max', verbose=1, patience=5, baseline=0.6) # stop when monitor is satisfact mode
                     # callbacks.ModelCheckpoint('best_model.h5', monitor='binary_accuracy', mode='max', save_best_only=True) # optimal pram save
          ])

Evaluate Model

In [ ]:
# Acc and Loss print
learning_lost, learning_acc = model.evaluate(X_train, y_train, verbose=2)
print("Learning accuracy :", learning_acc)
print("Learning loss % :", learning_lost)

# loss and acc graph (train and val)
history_df = pd.DataFrame(history.history)
history_df[["loss", "val_loss"]].plot()
history_df[["categorical_accuracy", "val_categorical_accuracy"]].plot()

Prediction

In [ ]:
Aengmu = model.predict(X_test)
print(Aengmu[0])
print(y_test[0])